## Import

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
#import matplotlib as mpl
import psycopg2
import pandas.io.sql as sqlio

import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns

from textwrap import wrap

In [2]:
con_osm = psycopg2.connect(database="osm", user="postgres", password="postgres", host="ma_db_01", port="5432")
con_postgres = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="ma_db_01", port="5432")
#autocommit mode - true
con_osm.autocommit = True
con_postgres.autocommit = True

## Modell Zugänglichkeit

Daten aus PostGIS DB laden

gemäß dem Modell die entsprechenden vier INPUT und eine OUTPUT Variable

In [3]:
sql = "SELECT fid, in_1, in_4, in_8, in_9, out FROM komm_flaechen_bbox;"

df_1 = sqlio.read_sql_query(sql, con_osm)
df_1.head(20)

,fid,in_1,in_4,in_8,in_9,out
0,634,0,0,0,0,0
1,15873,0,0,0,0,0
2,16744,0,0,0,0,1
3,2286,0,0,0,0,1
4,6468,0,0,0,0,1
5,16265,0,0,0,0,0
6,2273,0,0,0,0,0
7,15831,0,0,0,0,0
8,18355,0,0,0,0,0
9,23440,0,0,0,0,0


In [4]:
data = df_1[['in_1', 'in_4', 'in_8', 'in_9']]
data.head()

,in_1,in_4,in_8,in_9
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [5]:
target = df_1[['out']]
target.head()

,out
0,0
1,0
2,1
3,1
4,1


In [6]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(data, target.values.ravel())

LogisticRegression()

In [7]:
clf.score(data, target.values.ravel())

0.767432735426009

## Support Vector Classifier (SVC)

In [8]:
from sklearn.svm import SVC
model1 = SVC(kernel='linear', C=1)
model1.fit(data, target.values.ravel())
model1.score(data, target.values.ravel())

0.7675448430493274

## nicht-lineare Support-Vector Machine / Radial Basis

In [9]:
from sklearn.svm import SVC
model1 = SVC(kernel='rbf', C=1E3, gamma=1)
model1.fit(data, target.values.ravel())
model1.score(data, target.values.ravel())

0.7721973094170403

## Aufteilen in Trainingsdaten und Testdaten

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target.values.ravel(), 
                                                    test_size=0.30, 
                                                    random_state=42)


In [15]:
X_train

,in_1,in_4,in_8,in_9
7278,0,0,0,0
15609,0,0,0,0
1183,0,0,0,0
14144,0,0,0,0
9527,0,0,0,0
...,...,...,...,...
11284,0,0,0,0
11964,0,1,0,0
5390,0,0,0,0
860,0,0,0,0


In [12]:
y_train

array([0, 0, 0, ..., 1, 0, 1])

In [16]:
from sklearn.svm import SVC
model1 = SVC(kernel='linear', C=1)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.7698056801195815